<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/14_OpenLandMap_CLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract rainfall from OpenLandMap

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
aoi_name = "MKR_NS_buff_5km"

# GEE layer ID
layer_name = "RS_047"

## OpenLandMap Precipitation Monthly



In [ ]:
rainfall = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01") \
  .clip(aoi)

## Rainfall processing

In [ ]:
print(rainfall.getInfo())

band_list = rainfall.bandNames().getInfo()
print(type(band_list))
print(band_list)

## Export data - create task

In [ ]:
for i in band_list:
  output_img =  rainfall.select(i)
  output_name = f"{layer_name}_{aoi_name}_{i}"
  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 1000,
                                     maxPixels = 10e12
                                     )
  task.start()
  print("STARTED TASK ", i)

## Check task status

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:len(band_list)]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)